In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import RandomizedSearchCV, GroupKFold, train_test_split

In [2]:
df_plays = pd.read_csv('plays.csv')
df_plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8552,2021110100,4310,(1:56) (Shotgun) P.Mahomes sacked at NYG 16 fo...,4,3,8,KC,NYG,NYG,8,...,NaN,18.0,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,"1 DL, 3 LB, 7 DB",SCRAMBLE,0,Bracket,Other
8553,2021110100,4363,(1:07) (Shotgun) D.Jones pass short right to E...,4,1,10,NYG,KC,NYG,25,...,NaN,35.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE,0,Cover-2,Zone
8554,2021110100,4392,"(1:01) (No Huddle, Shotgun) D.Jones sacked at ...",4,2,7,NYG,KC,NYG,28,...,NaN,38.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone
8555,2021110100,4411,"(:39) (No Huddle, Shotgun) D.Jones pass incomp...",4,3,15,NYG,KC,NYG,20,...,NaN,30.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone


In [3]:
df_players = pd.read_csv('players.csv')
df_players.head()

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick


In [4]:
df_pffScouting = pd.read_csv('pffScoutingData.csv')
df_pffScouting.head()

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


In [5]:
# df_tracking = {}
df_tracking = pd.DataFrame()
weeks = range(1, 8)

# df_plays_week = []

for week in weeks:
    file = 'week' + str(week) + '.csv'
    df_tracking_temp = pd.read_csv(file)
#     for x in range(len(df_plays)):
#         count = 1
#         play = df_plays.iloc[x]
#         game_id = play.gameId
#         play_id = play.playId
#         if (df_tracking_temp.gameId[gameId].any() and df_tracking_temp.playId[playId].any()):
#             df_plays_week.append(week)
#         break
    df_tracking_temp['week'] = [week]*len(df_tracking_temp)
    df_tracking = pd.concat([df_tracking, df_tracking_temp]).reset_index(drop=True)

In [6]:
df_tracking

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,week
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,1
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,1
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,1
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,1
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335224,2021102500,3998,NaN,71,2021-10-26T03:17:05.900,NaN,football,left,112.93,14.52,22.04,0.44,2.62,NaN,NaN,pass_forward,7
7335225,2021102500,3998,NaN,72,2021-10-26T03:17:06.000,NaN,football,left,111.34,16.07,21.95,1.40,2.22,NaN,NaN,None,7
7335226,2021102500,3998,NaN,73,2021-10-26T03:17:06.100,NaN,football,left,109.74,17.60,21.81,2.02,2.21,NaN,NaN,None,7
7335227,2021102500,3998,NaN,74,2021-10-26T03:17:06.200,NaN,football,left,108.17,19.12,21.66,2.39,2.19,NaN,NaN,None,7


In [7]:
# weeks = df_tracking[['gameId', 'playId', 'week']]
# weeks = []
# for x in range(len(df_plays)):
#     play = df_plays.iloc[x]
#     gameId = play.gameId
#     playId = play.playId
    
# df_plays['week'] = np.where(((df_plays.gameId == df_tracking.gameId) & 
#                             (df_plays.playId == df_tracking.playId)), df_tracking.week, 0)

# df_plays.week
    
# print(weeks.shape)
# weeks = weeks.drop_duplicates()
# weeks = weeks.reset_index(drop=True)
# weeks

In [8]:
# df_plays = df_plays.merge(df_tracking[['gameId','playId', 'week']], on = ['gameId', 'playId'], how = 'left')
# df_plays

In [11]:
# Create the cache
dfs = {}

a = time.time()

# FOR EACH PLAY ADD THE RELEVANT TRACKING DATA INTO MEMORY
for x in range(len(df_plays)):
    play_id = df_plays.iloc[x]['playId']
    game_id = df_plays.iloc[x]['gameId']
    
    # Get the game tracking data
    gamedf = df_tracking[df_tracking.gameId == game_id]
    
    # Get the play tracking data within each game
    playdf = gamedf[gamedf.playId == play_id]
    
    # Data of moment where ball is snapped
    playdf_ball_snap = playdf[playdf.event == 'ball_snap']
    
    # Data of moment where pass is thrown or qb sack
    playdf_ball_thrown = playdf[(playdf.event == 'pass_forward') |
                                (playdf.event == 'qb_sack')]

    # Frame of when snap occurred
    frame_ball_snap = np.min(playdf_ball_snap.frameId)
    
#     if (len(playdf) == 0 or len(playdf_ball_thrown == 0)):
#         continue
    
    # Frame of when ball was thrown/sack/etc.
    frame_ball_thrown = np.min(playdf_ball_thrown.frameId)
    
    # Frames pre-snap and after-snap
    playdf_pre_snap = playdf[playdf.frameId <= frame_ball_snap]
    playdf_post_snap = playdf[playdf.frameId >= frame_ball_snap]
    
    # Frames pre-throw and post-throw
    playdf_before_throw = playdf[playdf.frameId <= frame_ball_thrown]
    playdf_after_throw = playdf[playdf.frameId >= frame_ball_thrown]

    # Frames of between snap and throw
    playdf_mid_play = playdf[(playdf.frameId <= frame_ball_thrown) & (playdf.frameId >= frame_ball_snap)]
    
    # Ball data
    playdf_ball = playdf[playdf.team == 'football']
    
    # SAVE THE DATA INTO THE CACHE FOR EACH TIMEFRAME
    dfs[str(game_id) + '--BALL+' + str(play_id)] = playdf_ball
    dfs[str(game_id) + '--FULL+' + str(play_id)] = playdf
    dfs[str(game_id) + '--ball_snap+' + str(play_id)] = playdf_ball_snap
    dfs[str(game_id) + '--pre_snap+' + str(play_id)] = playdf_pre_snap
    dfs[str(game_id) + '--post_snap+' + str(play_id)] = playdf_post_snap
    dfs[str(game_id) + '--mid_play+' + str(play_id)] = playdf_mid_play
    dfs[str(game_id) + '--before_throw+' + str(play_id)] = playdf_before_throw
    dfs[str(game_id) + '--after_throw+' + str(play_id)] = playdf_after_throw
    
b = time.time()

print(b-a)

112.85763192176819


In [13]:
## Feature Generation

offense_pos = set(['QB', 'RB', 'WR', 'TE', 'T', 'G', 'C', 'FB'])
defense_pos = set(['CB', 'DE', 'DT', 'FS', 'MLB', 'OLB', 'SS'])

def distance(X, Y):
    (x1, x2) = X
    (y1, y2) = Y
    return ((x1-x2)**2 + (y1-y2)**2) ** 0.5

def second_closest(vals):
    p1, p2 = float('inf'), float('inf')
    for x in vals:
        if x <= p1:
            p1, p2 = x, p1
        elif x < p2:
            p2 = x
    return p2

# time_intervals = time of [snap, throw]???
# def make_feature_rows(player_input, tracking_dict, time_intervals):

def make_feature_rows(player_input, tracking_dict):
    result = {}
    
    (pos, nflid, number) = player_input
    
    player_frames = tracking_dict[(pos, nflid, number)]
    
#     snap_frame, throw_frame = time_intervals[0], time_intervals[1]
    
#     times = [int(snap_frame/2), snap_frame, 
#              int((throw_frame - snap_frame)/3), 
#              int(2*((throw_frame - snap_frame)/3))]
    
#     ## Distance to nearest defensive players taken 4 times before throw is made
#     for x in times:
#         frame = player_frames[x]
#         player_x, player_y = frame[0], frame[1]
        
#         for key, val in tracking_dict.items():
#             if key == (pos, nflid, number):
#                 continue
#             if key[0] not in defense_pos:
#                 continue
#             if len(val) == 0:
#                 continue
#             p2_x, p2_y = val[x][0], val[x][1]
            
    ### OFFENSIVE PLAYER FEATURES
    
    # OFF_MEAN, OFF_VAR
    dist_nearest_off = []
    # RATIO OF DIST OF PLAYER TO CLOSEST OFFENSIVE PLAYER AND CLOSEST PLAYER TO SECOND CLOSEST
    dist_second_nearest_off = []
    # OFF_DIR_MEAN, OFF_DIR_MEAN
    nearest_off_dir = []
    # RATIO OF PLAYER TO CLOSEST OFFENSIVE PLAYER AND DIST TO CLOSEST DEF PLAYER
    nearest_off_xy = []
    for i in range(len(player_frames)):
        min_p = []
        min_dir = []
        min_xy = []
        for k, v in tracking_dict.items():
            if (k == (pos, nflid, number)): continue
            if (len(v) == 0): continue
            if (k[0] not in offense_pos): continue
            player = (player_frames[i][0], player_frames[i][1])
            other_player = (v[i][0], v[i][1])
            min_p.append(distance(player, other_player))
            min_dir.append(abs(player_frames[i][3]-v[i][3]))
            min_xy.append(other_player)
        dist_nearest_off.append(min(min_p))
        dist_second_nearest_off.append(second_closest(min_p))
        nearest_off_dir.append(min_dir[np.argmin(min_p)])
        nearest_off_xy.append(min_xy[np.argmin(min_p)])
    ratio = []
    for j in range(len(dist_nearest_off)):
        ratio_j = float(dist_nearest_off[j])/float(dist_second_nearest_off[j])
        ratio.append(ratio_j)
    result['OFF_MEAN'] = np.mean(dist_nearest_off)
    result['OFF_VAR'] = np.var(dist_nearest_off)
    result['OFF_DIR_MEAN'] = np.mean(nearest_off_dir)
    result['OFF_DIR_VAR'] = np.var(nearest_off_dir)
    result['OFF_RAT_MEAN'] = np.mean(ratio)
    result['OFF_RAT_VAR'] = np.var(ratio)
    result['OFF_RAT_SNAP'] = ratio[0]
    result['OFF_RAT_MID'] = ratio[(len(ratio)-1)//2]
    result['OFF_RAT_THROW'] = ratio[len(ratio)-1]
            
            
    ### DEFENSIVE PLAYER FEATURES
    # DEF_MEAN, DEF_VAR
    # DEF_DIR_MEAN, DEF_DIR_VAR
    # RATIO OF PLAYER TO CLOSEST DEF PLAYER AND SECOND CLOSEST DEF PLAYER
    # RATIO OF DIST TO CLOSEST DEF PLAYER AND DIST FROM CLOSEST DEF PLAYER TO CLOSEST OFF PLAYER
    dist_nearest_def = []
    dist_second_nearest_def = []
    nearest_def_dir = []
    nearest_def_xy = []
    for i in range(len(player_frames)):
        min_p = []
        min_dir = []
        min_xy = []
        for k, v in tracking_dict.items():
            if (k == (pos, nflid, number)): continue
            if (len(v) == 0): continue
            if (k[0] not in defense_pos): continue
            player = (player_frames[i][0], player_frames[i][1])
            other_player = (v[i][0], v[i][1])
            min_p.append(distance(player, other_player))
            min_dir.append(abs(player_frames[i][3]-v[i][3]))
            min_xy.append(other_player)
        dist_nearest_def.append(min(min_p))
        dist_second_nearest_def.append(second_closest(min_p))
        nearest_def_dir.append(min_dir[np.argmin(min_p)])
        nearest_def_xy.append(min_xy[np.argmin(min_p)])
    result['DEF_MEAN'] = np.mean(dist_nearest_def)
    result['DEF_VAR'] = np.var(dist_nearest_def)
    result['DEF_DIR_MEAN'] = np.mean(nearest_def_dir)
    result['DEF_DIR_VAR'] = np.var(nearest_def_dir)
        
    ratio_def = []
    ratio_def_off = []
    
    def_off_distances = []
    for x in range(len(nearest_def_xy)):
        dist = distance(nearest_def_xy[x],nearest_off_xy[x])
        def_off_distances.append(dist)

    for j in range(len(dist_nearest_def)):
        ratio_j = float(dist_nearest_def[j])/float(dist_second_nearest_def[j])
        if def_off_distances[j] == 0:
            ratio_d = 0
        else:
            ratio_d = float(dist_nearest_def[j])/float(def_off_distances[j])
        ratio_def.append(ratio_j)
        ratio_def_off.append(ratio_d)
        
    result['DEF_RAT_MEAN'] = np.mean(ratio_def)
    result['DEF_RAT_VAR'] = np.var(ratio_def)
    result['DEF_RAT_SNAP'] = ratio_def[0]
    result['DEF_RAT_MID'] = ratio_def[(len(ratio_def)-1)//2]
    result['DEF_RAT_THROW'] = ratio_def[len(ratio_def)-1]
    
    result['DEF_OFF_RAT_MEAN'] = np.mean(ratio_def_off)
    result['DEF_OFF_RAT_VAR'] = np.var(ratio_def_off)
    result['DEF_OFF_RAT_SNAP'] = ratio_def_off[0]
    result['DEF_OFF_RAT_MID'] = ratio_def_off[(len(ratio_def_off)-1)//2]
    result['DEF_OFF_RAT_THROW'] = ratio_def_off[len(ratio_def_off)-1]
    
    ### PLAYER VALUES
    ## VARIANCE OF X FOR PLAYER
    ## VARIANCE OF Y FOR PLAYER
    ## VARIANCE OF SPEED FOR PLAYER
    var_x = np.var([x[0] for x in player_frames])
    var_y = np.var([x[1] for x in player_frames])
    var_s = np.var([x[2] for x in player_frames])
    result['VAR_X'] = var_x
    result['VAR_Y'] = var_y
    result['VAR_S'] = var_s
        
        
    if(len(result.keys()) == 0): print("EMPTY")
    return result

In [36]:
## Creating input data to create features

to_df = []

# offensive_line = ['T', 'G', 'C']
linebackers = ['ILB', 'OLB']

for x in range(len(df_plays)):
    play = df_plays.iloc[x]
    play_id = play['playId']
    game_id = play['gameId']

    df_ball = dfs[str(game_id) + '--BALL+' + str(play_id)]
    df_play = dfs[str(game_id) + '--mid_play+' + str(play_id)]

    prev = None
    prev_num = None
    curr = []
    pos_dict = {}


    for index, row in df_play.iterrows():
        if(row['team'] == 'football'): continue
        if(row['nflId'] == prev):
            curr.append((row['x'],row['y'],row['s'],row['dir']))
        else:
            if(prev != None):
                position = (df_players[df_players['nflId'] == prev]).iloc[0]['officialPosition']
                pos_dict[(position,prev,prevNum)] = curr
            prev = row['nflId']
            prevNum = row['jerseyNumber']
            curr = []
            curr = [(row['x'],row['y'],row['s'],row['dir'])]
    
    ### FIND THE O-LINEMEN AND THEN CREATE FEATURES FOR THEM ###
    for k , v in pos_dict.items(): 
        if(k[0] in linebackers):
#         if(k[0] in offensive_line):
            
            if(len(pos_dict[k]) == 0): continue
            featurerow = make_feature_rows(k,pos_dict)
            featurerow['gameId'] = game_id
            featurerow['playId'] = play_id
            featurerow['position'] = k[0]
            featurerow['nflId'] = k[1]
            featurerow['number'] = k[2]
            featurerow['yardlineNumber'] = play['yardlineNumber']
            featurerow['yardlineSide'] = play['yardlineSide']
            to_df.append(featurerow)

final_df = pd.DataFrame(to_df)

In [37]:
final_df.iloc[:, :-8]

,OFF_MEAN,OFF_VAR,OFF_DIR_MEAN,OFF_DIR_VAR,OFF_RAT_MEAN,OFF_RAT_VAR,OFF_RAT_SNAP,OFF_RAT_MID,OFF_RAT_THROW,DEF_MEAN,...,DEF_RAT_SNAP,DEF_RAT_MID,DEF_RAT_THROW,DEF_OFF_RAT_MEAN,DEF_OFF_RAT_VAR,DEF_OFF_RAT_SNAP,DEF_OFF_RAT_MID,DEF_OFF_RAT_THROW,VAR_X,VAR_Y
0,17.224223,10.863352,236.162857,1009.811718,0.910803,0.004411,0.992481,0.924554,0.775573,17.665132,...,0.969808,0.969108,0.842636,2.230654,0.221537,3.066787,1.901151,2.255213,9.113865,0.162419
1,105.362467,0.468994,72.721923,277.243792,0.961740,0.000041,0.950700,0.961852,0.968804,100.664613,...,0.931644,0.937596,0.947332,1.230831,0.005259,1.157138,1.202739,1.381903,0.875961,8.389837
2,91.508131,25.209862,89.515000,4966.361056,0.950518,0.000018,0.941863,0.951932,0.950641,86.133316,...,0.917915,0.920055,0.882275,1.050555,0.000082,1.044101,1.047495,1.069805,0.242261,7.577369
3,95.072119,17.169762,36.683462,564.414715,0.953906,0.000021,0.944790,0.954718,0.957159,89.895456,...,0.922518,0.925041,0.927545,1.097095,0.000547,1.078763,1.086225,1.159493,2.112974,0.491481
4,62.468067,0.072879,179.239565,369.026161,0.973497,0.000013,0.970974,0.973933,0.972158,56.868446,...,0.931690,0.943809,0.969503,1.258072,0.020716,1.099326,1.212664,1.546845,5.863411,0.040248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17022,73.801591,13.528683,53.283684,1612.630576,0.935106,0.000024,0.930718,0.941435,0.928467,70.246816,...,0.896943,0.858777,0.770060,1.450764,0.070777,1.217322,1.332388,2.150386,12.708078,13.529198
17023,80.334431,11.936013,183.277500,3911.808923,0.935068,0.000012,0.933749,0.938661,0.931627,79.114408,...,0.945284,0.906535,0.933358,1.892436,0.386185,1.242571,1.641661,3.188987,23.905395,7.328491
17024,68.317324,159.744475,150.688077,8081.373481,0.904416,0.001686,0.935776,0.929551,0.819045,67.270729,...,0.947081,0.892665,0.832809,1.523222,0.031680,1.265315,1.514004,1.845073,0.890514,60.250127
17025,70.094238,149.547071,50.341875,4326.996790,0.922291,0.001476,0.977140,0.921189,0.844015,63.428072,...,0.997891,0.979488,0.875056,1.072559,0.016457,0.879418,1.072255,1.268984,8.866927,2.201425


In [38]:
gmm = GaussianMixture(n_components=3)

gmm_df = final_df.iloc[:, :-8]

#FIT THE DATA TO THE GMM#
preds = gmm.fit(gmm_df)

#GET THE PREDICTION PROBABILITIES#
probs = gmm.predict_proba(gmm_df)

In [39]:
def get_assignments(row):
    cluster = np.argmax(row)
    return 'cluster ' + str(cluster)

results = pd.DataFrame(data = probs)
preds = results.apply(get_assignments, axis = 1)

final_df['pred'] = preds
final_df.head(50)

,OFF_MEAN,OFF_VAR,OFF_DIR_MEAN,OFF_DIR_VAR,OFF_RAT_MEAN,OFF_RAT_VAR,OFF_RAT_SNAP,OFF_RAT_MID,OFF_RAT_THROW,DEF_MEAN,...,VAR_Y,VAR_S,gameId,playId,position,nflId,number,yardlineNumber,yardlineSide,pred
0,17.224223,10.863352,236.162857,1009.811718,0.910803,0.004411,0.992481,0.924554,0.775573,17.665132,...,0.162419,0.788183,2021090900,97,OLB,53441.0,11.0,33,TB,cluster 1
1,105.362467,0.468994,72.721923,277.243792,0.961740,0.000041,0.950700,0.961852,0.968804,100.664613,...,8.389837,3.338358,2021090900,137,ILB,38588.0,54.0,2,DAL,cluster 0
2,91.508131,25.209862,89.515000,4966.361056,0.950518,0.000018,0.941863,0.951932,0.950641,86.133316,...,7.577369,4.693006,2021090900,137,OLB,41915.0,58.0,2,DAL,cluster 0
3,95.072119,17.169762,36.683462,564.414715,0.953906,0.000021,0.944790,0.954718,0.957159,89.895456,...,0.491481,1.612349,2021090900,137,ILB,47788.0,45.0,2,DAL,cluster 0
4,62.468067,0.072879,179.239565,369.026161,0.973497,0.000013,0.970974,0.973933,0.972158,56.868446,...,0.040248,2.927387,2021090900,187,ILB,38588.0,54.0,34,DAL,cluster 0
5,55.433367,14.353112,139.964783,9049.990425,0.966625,0.000017,0.968044,0.968543,0.954136,49.069324,...,4.073029,3.980847,2021090900,187,OLB,41915.0,58.0,34,DAL,cluster 0
6,57.827894,5.277969,49.631739,767.161553,0.969349,0.000010,0.968901,0.970291,0.962813,51.753577,...,0.078025,0.291225,2021090900,187,ILB,47788.0,45.0,34,DAL,cluster 0
7,21.846708,3.008949,76.259091,9169.535596,0.951677,0.000547,0.963827,0.920321,0.944219,13.302723,...,0.162045,0.114328,2021090900,282,ILB,38588.0,54.0,39,TB,cluster 0
8,26.803776,16.926166,47.348485,5005.482558,0.965798,0.000282,0.966570,0.946336,0.971782,20.498409,...,2.359482,0.812024,2021090900,282,OLB,41915.0,58.0,39,TB,cluster 0
9,25.916640,5.143886,103.933030,1174.970045,0.964361,0.000335,0.974096,0.938575,0.967890,19.298877,...,5.178297,2.837471,2021090900,282,ILB,47788.0,45.0,39,TB,cluster 0
